<a href="https://colab.research.google.com/github/Sid-202/ML-Project-Customer-Goodness-Credit-/blob/main/customer_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>